# How-to train a model on Azure ML

This notebook takes you through the steps of training a model on Azure ML for The Ocean Cleanup. We train the models through Azure ML to provide us with a good registration of all performed tests, so that we can see why and how a model was created.

When the result of a training run is satisfactory, a model can be registered from there, from which point we can deploy it.

There are a few concepts to know about first:

- Workspace: The entire AzureML environment you are working in. The Workspace contains all the other elements.
- Experiment: A collection of Runs (see below). A logical container for training a model with different parameters to determine the best.
- Run: A single train/test run of a model. These are tied to an experiment. If you want to train the same model with different parameters, so you can compare them, these are different runs under the same experiment.
- Environment: The code environment used by your code. This contains things like the required Python packages. Multiple options exist here - from just using your local environment to completely curated environments directly from Azure.
- Dataset: A single dataset as registered in the AzureML workspace.

With that out of the way, lets dive right in. Looking at these components, our first step will be to get the correct Workspace:

In [1]:
from azurewrapper.workspace import get_workspace

subscription_id = "29d66431-a7ce-4709-93f7-3bdb01a243b3"
resource_group = "ExperimentationJayke"
workspace_name = "ExperimentationJayke"

workspace = get_workspace(subscription_id, resource_group, workspace_name)

## Create experiment

Now that we have a workspace available, we need to create an experiment. As describe above, an experiment will be the container for multiple runs, in which we can train and compare the model using different parameters.

The experiment needs a name. Use something that is descriptive and clear to anyone seeing this.

In [2]:
from azurewrapper.train import create_experiment
experiment = create_experiment(workspace, "model-frcnn-v-1-0")

## Create or select compute target

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
cluster_name = "gpu-cluster"

try:
    compute_target = ComputeTarget(workspace=workspace, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

Creating a new compute target...
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Create the environment

We will now need to create an environment. In this case, we use the curated TensorFlow 2.2 environment.

In [4]:
from azureml.core import Environment
from azurewrapper.environment import get_environment
# environment = get_environment(
#     workspace,
#     "AzureML-TensorFlow-2.2-GPU"
# )

In [5]:
environment = Environment.from_conda_specification(
    "custom_tensorflow_object_detection",
    file_path='examples/frcnn/conda_dependencies.yml'
)
environment.docker.enabled = True


# Load from dockerfile
environment.docker.base_image = None
environment.docker.base_dockerfile = "./examples/frcnn/Dockerfile"
environment.python.user_managed_dependencies = True

In [6]:

# tf_env = Environment.from_conda_specification(
#     name='tensorflow-2.2-gpu',
#     file_path='./examples/frcnn/conda_dependencies.yml'
# )

# # Specify a GPU base image
# tf_env.docker.enabled = True
# tf_env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04'
# environment = tf_env

## Prepare model wraper

Now it's time to perform our first Run of the experiment. However, before we can do this, we will need a wrapper around our model. This wrapper needs to do a few things:

- Initialize and train the model with:
  - The desired parameters
  - The desired data
- Evaluate the performance of the trained model
- Register the parameters and the performance in the Run object
- Add the generated model artifacts to the Run object

There is skeleton code for this available: `skeleton_files/train.py`. In this file you fill in what parameters you expect, you create and train and evaluate the model using these parameters and the loaded in dataset(s), and you register the results and the created artifacts with the Run.

For this how-to, we will use the example provided in `examples/frcnn/train.py`. This is an implementation of the file mentioned above. It expects two parameters: `num_train_steps` and `sample_1_of_n_eval_examples`.

## Run the experiment

Now we need to create and run the experiment. First, we fetch the desired datasets, and combine these into train- and test sets. Note that we can provide multiple sets for both training and testing. Also note that each set consists of both a label and an image dataset.

In [7]:
from azureml.core import Dataset

train_images = Dataset.get_by_name(workspace, name="campaign-26-10-2020_images")
train_labels = Dataset.get_by_name(workspace, name="campaign-26-10-2020_labels")
test_images = Dataset.get_by_name(workspace, name="campaign-22-10-2020_images")
test_labels = Dataset.get_by_name(workspace, name="campaign-22-10-2020_labels")
trainsets = [
    (train_labels, train_images),
    (test_labels, test_images)
]
testsets = [
    (test_labels, test_images)
]

We now have everything we need to perform the run locally. Lets do so!

In [9]:
from azurewrapper.train import perform_run
from azureml.core.runconfig import TensorflowConfiguration

checkpoint_files = Dataset.get_by_name(workspace, name="FRCNN")

# perform_run(experiment, script, source_directory, environment=None,
#             compute_target=None, datasets=[], parameters={})
# run = perform_run(experiment, 'train.py', 'examples/example_model', environment=environment,
#                   datasets=datasets, parameters={'param_a': 30, 'param_b': 12.0})
run = perform_run(experiment, 'train.py', 'examples/frcnn', environment=environment,
                  trainsets=trainsets, testsets=testsets, compute_target=compute_target,
                  parameters={
                      'num_train_steps': 10000,
                      'sample_1_of_n_eval_examples': 1,
                      'checkpoint_dataset': checkpoint_files.as_named_input(f'checkpoint').as_mount()
                  })
run.wait_for_completion(show_output=True)

RunId: model-frcnn-v-1-0_1604312174_b7a3065b
Web View: https://ml.azure.com/experiments/model-frcnn-v-1-0/runs/model-frcnn-v-1-0_1604312174_b7a3065b?wsid=/subscriptions/29d66431-a7ce-4709-93f7-3bdb01a243b3/resourcegroups/ExperimentationJayke/workspaces/ExperimentationJayke

Streaming azureml-logs/20_image_build_log.txt

2020/11/02 10:16:20 Downloading source code...
2020/11/02 10:16:21 Finished downloading source code
2020/11/02 10:16:23 Creating Docker network: acb_default_network, driver: 'bridge'
2020/11/02 10:16:24 Successfully set up Docker network: acb_default_network
2020/11/02 10:16:24 Setting up Docker configuration...
2020/11/02 10:16:24 Successfully set up Docker configuration
2020/11/02 10:16:24 Logging in to registry: 4974f70cd2934b4299204f2bf3475cda.azurecr.io
2020/11/02 10:16:26 Successfully logged into 4974f70cd2934b4299204f2bf3475cda.azurecr.io
2020/11/02 10:16:26 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2

  Created wheel for json-logging-py: filename=json_logging_py-0.2-cp37-none-any.whl size=3924 sha256=47c838d49b20383986c8d8c795eba84e96401c767cbe89113e90fddc97a0d640
  Stored in directory: /root/.cache/pip/wheels/0d/2e/1c/c638b7589610d8b9358a6e5eb008edacb8b3e9b6d1edc9479f
  Created wheel for termcolor: filename=termcolor-1.1.0-cp37-none-any.whl size=4832 sha256=148aa9fef6c4dd21ef5b75f990f3a9f17ca64416df3e0b6af0225f48c52ad889
  Stored in directory: /root/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=70936 sha256=d742ac393dc14c48e35d097baa612ff897d35b204aaebf9605c14cf2c0bd0f26
  Stored in directory: /root/.cache/pip/wheels/b1/c2/ed/d62208260edbd3fa7156545c00ef966f45f2063d0a84f8208a
  Created wheel for liac-arff: filename=liac_arff-2.5.0-cp37-none-any.whl size=11732 sha256=260ea8c18741acd4d1c91e0fddb2f5b50aed7f54c6b29d48629d1880d6961a27
  Stored in directory: /root/.cache/pip/

Removing intermediate container c66931db7676
 ---> f8036a506128
Step 3/26 : RUN mkdir /install
 ---> Running in 500483aa58cc
Removing intermediate container 500483aa58cc
 ---> ba3628e6f014
Step 4/26 : RUN mkdir /install/TensorFlow
 ---> Running in fdbdebf868cf
Removing intermediate container fdbdebf868cf
 ---> 4dcfc2ac242b
Step 5/26 : WORKDIR /install/TensorFlow
 ---> Running in afe8390f7d27
Removing intermediate container afe8390f7d27
 ---> 25555120e012
Step 6/26 : RUN wget https://github.com/tensorflow/models/archive/master.zip
 ---> Running in aa4e878399e0
--2020-11-02 10:22:26--  https://github.com/tensorflow/models/archive/master.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/tensorflow/models/zip/master [following]
--2020-11-02 10:22:26--  https://codeload.github.com/tensorflow/models/zip/master
Resolving codeload.git

  Created wheel for object-detection: filename=object_detection-0.1-cp37-none-any.whl size=1597832 sha256=bf8e2f3596095c588473f06e96ef135fa0d35356aef4d0850c519a70f8a743e2
  Stored in directory: /tmp/pip-ephem-wheel-cache-8n1pd0f1/wheels/9a/05/1a/ee445bde6c89c43de64a08ce8830423e2af1082f7cdda0ad61
  Created wheel for avro-python3: filename=avro_python3-1.10.0-cp37-none-any.whl size=43734 sha256=30797607abf3fc67c95531e8a2c35dd1c5e0faa24f8bb300e57d7d5e3a3dc8f7
  Stored in directory: /root/.cache/pip/wheels/3f/15/cd/fe4ec8b88c130393464703ee8111e2cddebdc40e1b59ea85e9
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=292025 sha256=4b06a1f70feae91dada2a5b2ed0358b781b2eb122ee723dd52d2c863865b9d7a
  Stored in directory: /root/.cache/pip/wheels/68/a5/e7/56401832f23d0b2db351c5b682e466cb4841960b086da65e4e
  Created wheel for dill: filename=dill-0.3.1.1-cp37-none-any.whl size=78533 sha256=656c311b375d8de6a230b9cf64e4dce0e62905360d0dfb034b73cfd0bc41c092
  St

Removing intermediate container 752509ac6172
 ---> 33678cafc065
Step 18/26 : WORKDIR /
 ---> Running in 4ebd28f0b089
Removing intermediate container 4ebd28f0b089
 ---> 2614ff679fd7
Step 19/26 : USER root
 ---> Running in fce9da0b2485
Removing intermediate container fce9da0b2485
 ---> 905d604b2603
Step 20/26 : RUN mkdir -p $HOME/.cache
 ---> Running in 087dbc689125
Removing intermediate container 087dbc689125
 ---> 270b1ffa80e0
Step 21/26 : WORKDIR /
 ---> Running in c0305a715624
Removing intermediate container c0305a715624
 ---> aa2162b6b1dc
Step 22/26 : COPY azureml-environment-setup/99brokenproxy /etc/apt/apt.conf.d/
 ---> 7f488fcf7a14
Step 23/26 : COPY azureml-environment-setup/spark_cache.py azureml-environment-setup/log4j.properties /azureml-environment-setup/
 ---> bd50a1ef8ada
Step 24/26 : RUN if [ $SPARK_HOME ]; then /bin/bash -c '$SPARK_HOME/bin/spark-submit  /azureml-environment-setup/spark_cache.py'; fi
 ---> Running in edae68e82903
Removing intermediate container edae68e829

ee06e59a4314: Pull complete
b0ac690bb2f2: Pull complete
5f11ace0944e: Pull complete
d4e18c29e104: Pull complete
09ba06861fca: Pull complete
83213e0b5131: Pull complete
7b0feb8d64db: Pull complete
7b0b507bb1e9: Pull complete
94e0f0e2d7da: Pull complete
45b4bc5400c1: Pull complete
30f9a1497794: Pull complete
9a7ac860589c: Pull complete
0f38927fa8e0: Pull complete
1d7ed14a884d: Pull complete
c01c1fe8898f: Pull complete
81dc248a70ed: Pull complete
cf550e0b0dc2: Pull complete
cf23e4a43291: Pull complete
71780e028e5f: Pull complete
2c16f1517937: Pull complete
142c5579ef76: Pull complete
feed4966b399: Pull complete
80d2d9c434de: Pull complete
36cd24390d80: Pull complete
6e6ad0ed3b63: Pull complete
Digest: sha256:fdbdb69fdfa131fbb95dd0d701ab5c2627908fc8fd5dc081e26f02d584a1e90a
Status: Downloaded newer image for 4974f70cd2934b4299204f2bf3475cda.azurecr.io/azureml/azureml_ac7d441ea8eec91eff9e12f161d95946:latest
c1697d9da5f2ca2d548d3589866b7cfbcdc65d64d1a3678a27b90e029aadacca
2020/11/02 10:40:40 


Streaming azureml-logs/75_job_post-tvmps_83e953c61b6c30dfa0876ed2be63bdd023297f0007de806b2d24eb05dbbf971c_d.txt

Entering job release. Current time:2020-11-02T10:41:48.596568
Starting job release. Current time:2020-11-02T10:41:49.338818
Logging experiment finalizing status in history service.
[2020-11-02T10:41:49.339436] job release stage : upload_datastore starting...
Starting the daemon thread to refresh tokens in background for process with pid = 924
[{}] job release stage : start importing azureml.history._tracking in run_history_release.
[2020-11-02T10:41:49.341756] job release stage : copy_batchai_cached_logs starting...[2020-11-02T10:41:49.341794] job release stage : execute_job_release starting...
[2020-11-02T10:41:49.341924] job release stage : copy_batchai_cached_logs completed...

[2020-11-02T10:41:49.342377] Entering context manager injector.
[2020-11-02T10:41:49.349479] job release stage : upload_datastore completed...
[2020-11-02T10:41:49.553361] job release stage : send

{'runId': 'model-frcnn-v-1-0_1604312174_b7a3065b',
 'target': 'gpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2020-11-02T10:37:49.423019Z',
 'endTimeUtc': '2020-11-02T10:42:00.573015Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'f67dc28a-b753-4683-b8da-f70e73c644db',
  'azureml.git.repository_uri': 'git@github.com:TheOceanCleanup/AIDataPipeLine.git',
  'mlflow.source.git.repoURL': 'git@github.com:TheOceanCleanup/AIDataPipeLine.git',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': '7b1ff496025f18b292f735b470fb28827a3bb878',
  'mlflow.source.git.commit': '7b1ff496025f18b292f735b470fb28827a3bb878',
  'azureml.git.dirty': 'True',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '95c95241-ca72-45cd-aeb6-106bb9c2fd40'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'checkpoint', 'mecha